# Batch Correction for TCGA DNA Methylation Data
In this notebook, I want to remove batch effects from the previously downloaded DNA methylation data.
The obtained data was already preprocessed such that we have two sample matrices $S_{tumor} \in \mathbb{R}^{N \times M_1}$ and $S_{normal} \in \mathbb{R}^{N \times M_2}$ with $M_1$ and $M_2$ being the number of samples for tumor and normal tissues across all cancer types and $N$ being the number of genes.

**That is, we already have computed the average promoter DNA methylation across all measured CpG sites per gene.**

Next, we want to remove the batch effects using *ComBat*. As batches, we use the plate IDs as suggested in multiple articles.

The workflow of that notebook is as follows:
1. Load the big sample matrices for tumor and normal
2. Split them into smaller gene-sample matrices for each cancer type and write them to disk
3. Call a R script which does the batch correction with ComBat

The results from batch correction are then read by another script and the final feature matrix is computed there.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib import gridspec
plt.rc('font', family='Times New Roman')
import h5py
import seaborn as sns
import os

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

%matplotlib inline

## 1. Load Tumor and Normal Sample Matrices

In [2]:
tumor_samples = pd.read_csv('../data/pancancer/TCGA/DNA_methylation/dna_methy_matrix_tumor.tsv',
                               compression='gzip',
                               sep='\t')
# rename first column to 'Symbol'
tumor_samples.columns = ['Symbol'] + tumor_samples.columns[1:].tolist()
# get rid of all the support columns
tumor_betaval_cols = ['Symbol'] + [i for i in tumor_samples.columns[1:] if "mean_beta_value_promoter" in i]
tumor_samples = tumor_samples[tumor_betaval_cols]
# put the symbol as index
tumor_samples.set_index('Symbol', inplace=True)
# get cancer type, transform and add it as column
ctype_tumor = [i.split('|')[1].upper() for i in tumor_samples.columns]
tumor_samples_t = tumor_samples.T
tumor_samples_t['ctype'] = ctype_tumor
tumor_samples.head()

,TCGA-4Z-AA87-01A-11D-A392-05|blca|mean_beta_value_promoter,TCGA-GU-A766-01A-11D-A32C-05|blca|mean_beta_value_promoter,TCGA-GU-A767-01A-11D-A32C-05|blca|mean_beta_value_promoter,TCGA-BT-A0S7-01A-11D-A10W-05|blca|mean_beta_value_promoter,TCGA-XF-A9SV-01A-21D-A42F-05|blca|mean_beta_value_promoter,TCGA-4Z-AA7O-01A-31D-A392-05|blca|mean_beta_value_promoter,TCGA-XF-AAN8-01A-11D-A42F-05|blca|mean_beta_value_promoter,TCGA-H4-A2HO-01A-11D-A17Y-05|blca|mean_beta_value_promoter,TCGA-DK-A1AC-01A-11D-A13Z-05|blca|mean_beta_value_promoter,TCGA-XF-A9SL-01A-11D-A392-05|blca|mean_beta_value_promoter,...,TCGA-GV-A40E-01A-12D-A23O-05|blca|mean_beta_value_promoter,TCGA-XF-A9SI-01A-11D-A392-05|blca|mean_beta_value_promoter,TCGA-BT-A0YX-01A-11D-A10W-05|blca|mean_beta_value_promoter,TCGA-ZF-AA4U-01A-11D-A38H-05|blca|mean_beta_value_promoter,TCGA-XF-AAMW-01A-11D-A42F-05|blca|mean_beta_value_promoter,TCGA-BT-A42C-01A-11D-A23O-05|blca|mean_beta_value_promoter,TCGA-E5-A4U1-01A-11D-A31M-05|blca|mean_beta_value_promoter,TCGA-G2-A3IB-01A-11D-A211-05|blca|mean_beta_value_promoter,TCGA-XF-AAN7-01A-11D-A42F-05|blca|mean_beta_value_promoter,TCGA-GV-A6ZA-01A-12D-A33I-05|blca|mean_beta_value_promoter
Symbol,,,,,,,,,,,,,,,,,,,,,
A1BG,0.766182,0.879027,0.638150,0.539798,0.569204,0.307855,0.853493,0.410151,0.471452,0.755809,...,0.472618,0.661983,0.621587,0.161164,0.466445,0.395604,0.215799,0.247464,0.159511,0.317184
A1BG-AS1,0.098877,0.063922,0.152120,0.043820,0.248357,0.027743,0.103687,0.039811,0.310905,0.109133,...,0.135036,0.051697,0.168068,0.033810,0.077383,0.256101,0.056729,0.072079,0.122777,0.190172
A1CF,0.414742,0.668591,0.608246,0.622058,0.409986,0.260519,0.676232,0.436054,0.440435,0.758311,...,0.643717,0.747106,0.668924,0.299899,0.444741,0.478200,0.261600,0.244479,0.307037,0.465353
A2M,0.283542,0.496870,0.640682,0.522662,0.508536,0.450079,0.504712,0.701825,0.632247,0.459291,...,0.311517,0.502696,0.437635,0.291915,0.502368,0.704262,0.676290,0.392833,0.404207,0.610936
A2ML1,0.401354,0.742049,0.404987,0.555399,0.208200,0.239268,0.867152,0.380197,0.394116,0.773035,...,0.438861,0.624627,0.481523,0.184431,0.352999,0.349068,0.199723,0.186643,0.641958,0.289543


In [3]:
normal_samples = pd.read_csv('../data/pancancer/TCGA/DNA_methylation/dna_methy_matrix_normal.tsv',
                             compression='gzip',
                             sep='\t')
# put symbol in first column name
normal_samples.columns = ['Symbol'] + normal_samples.columns[1:].tolist()
# get rid of all the support columns
normal_betaval_cols = ['Symbol'] + [i for i in normal_samples.columns[1:] if "mean_beta_value_promoter" in i]
normal_samples = normal_samples[normal_betaval_cols]
# put the symbol as index
normal_samples.set_index('Symbol', inplace=True)
# get cancer type, transform and add it as column
ctype_normal = [i.split('|')[1].upper() for i in normal_samples.columns]
normal_samples_t = normal_samples.T
normal_samples_t['ctype'] = ctype_normal
normal_samples_t.head()

Symbol,A1BG,A1BG-AS1,A1CF,A2M,A2ML1,A4GNT,AAAS,AACS,AADAC,AADACL2,...,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,yR211F11.2,ctype
TCGA-GD-A3OP-11A-11D-A223-05|blca|mean_beta_value_promoter,0.731775,0.059683,0.742097,0.560357,0.578255,0.684671,0.248734,0.063237,0.884721,0.799980,...,0.512314,0.462903,0.111976,0.399045,0.051469,0.040117,0.072926,0.033753,0.912519,BLCA
TCGA-CU-A0YR-11A-13D-A10W-05|blca|mean_beta_value_promoter,0.740044,0.061980,0.793657,0.403698,0.802171,0.697633,0.174140,0.056394,0.828316,0.853067,...,0.069025,0.258704,0.100775,0.328573,0.052909,0.055996,0.083898,0.039314,0.874165,BLCA
TCGA-BT-A20U-11A-11D-A14Z-05|blca|mean_beta_value_promoter,0.696067,0.181428,0.755400,0.514972,0.613322,0.647983,0.176538,0.075458,0.844293,0.856609,...,0.497523,0.453023,0.117964,0.530543,0.073868,0.066610,0.100492,0.045037,0.880080,BLCA
TCGA-BT-A20N-11A-11D-A14Z-05|blca|mean_beta_value_promoter,0.741108,0.072333,0.874254,0.396894,0.861094,0.787942,0.200571,0.067082,0.886469,0.916663,...,0.097017,0.214441,0.103331,0.314471,0.056530,0.068143,0.089264,0.045082,0.902064,BLCA
TCGA-GD-A2C5-11A-11D-A17Y-05|blca|mean_beta_value_promoter,0.609897,0.048246,0.817458,0.568332,0.644737,0.583183,0.289412,0.067387,0.872940,0.841379,...,0.462548,0.458482,0.122215,0.240333,0.066481,0.044290,0.083030,0.037566,0.906742,BLCA


## 2. Split per Cancer Type and write small matrices to disk

In [4]:
cols_of_interest = [i for i in tumor_samples.columns[1:] if i.split('|')[1] == 'blca']
tumor_samples[cols_of_interest].shape

(28353, 418)

In [5]:
base_dir = '../data/pancancer/TCGA/DNA_methylation/'

processed_cancertypes = []
for ctype in tumor_samples_t.ctype.unique():
    # tumor samples
    cols_of_interest = [i for i in tumor_samples.columns if i.split('|')[1].upper() == ctype]
    tumor_samples_ctype = tumor_samples[cols_of_interest]
    
    # normal samples
    cols_of_interest = [i for i in normal_samples.columns if i.split('|')[1].upper() == ctype]
    normal_samples_ctype = normal_samples[cols_of_interest]
    
    # don't process the cancer type when there are no samples
    if tumor_samples_ctype.shape[1] <= 1 or normal_samples_ctype.shape[1] <= 1:
        print ("Warning: No normal or tumor data for {}... Not processing it".format(ctype))
        continue

    # directory
    ctype_dir = os.path.join(base_dir, ctype)
    if not os.path.isdir(ctype_dir):
        os.mkdir(ctype_dir)

    # write them to disk
    tumor_samples_ctype.dropna(axis=0).to_csv(os.path.join(ctype_dir, 'tumor_samples.tsv'), sep='\t')
    normal_samples_ctype.dropna(axis=0).to_csv(os.path.join(ctype_dir, 'normal_samples.tsv'), sep='\t')
    
    # pheno data
    pheno_t = pd.DataFrame(tumor_samples_ctype.columns, columns=['Name'])
    pheno_t['index'] = np.arange(pheno_t.shape[0])
    pheno_t['cancer'] = 'tumor'
    pheno_t['batch'] = [i[0].split('-')[5] for i in pheno_t.Name.str.split('|')]
    pheno_t.set_index('Name', inplace=True)
    pheno_t.to_csv(os.path.join(ctype_dir, 'pheno_tumor.tsv'), sep='\t')

    pheno_n = pd.DataFrame(normal_samples_ctype.columns, columns=['Name'])
    pheno_n['index'] = np.arange(pheno_n.shape[0])
    pheno_n['cancer'] = 'normal'
    pheno_n['batch'] = [i[0].split('-')[5] for i in pheno_n.Name.str.split('|')]
    pheno_n.set_index('Name', inplace=True)
    pheno_n.to_csv(os.path.join(ctype_dir, 'pheno_normal.tsv'), sep='\t')

    print ("Wrote matrices for cancer type {}".format(ctype))
    processed_cancertypes.append(ctype)

Wrote matrices for cancer type BLCA


## 3. Do batch correction

In [6]:
import subprocess

base_path = '../data/pancancer/TCGA/DNA_methylation/{}'
call = 'Rscript batch_correction.R {} {} {}'
for ctype in processed_cancertypes:
    ctype_dir = base_path.format(ctype)
    tumor_sample_path = os.path.join(ctype_dir, 'tumor_samples.tsv')
    tumor_pheno_path = os.path.join(ctype_dir, 'pheno_tumor.tsv')
    tumor_out_path = os.path.join(ctype_dir, 'tumor_samples.adjusted.tsv')
    subprocess.call(call.format(tumor_sample_path, tumor_pheno_path, tumor_out_path), shell=True)

    normal_sample_path = os.path.join(ctype_dir, 'normal_samples.tsv')
    normal_pheno_path = os.path.join(ctype_dir, 'pheno_normal.tsv')
    normal_out_path = os.path.join(ctype_dir, 'normal_samples.adjusted.tsv')
    subprocess.call(call.format(normal_sample_path, normal_pheno_path, normal_out_path), shell=True)
    print ("Processed {}".format(ctype))

Processed BLCA
